# * VINSIGHT : CEO Report

In [1]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Import Transaction

In [2]:
''' Execute transaction '''


# Input parameter
v_start_date = 20240101
print(f'\nParameter input...')
print(f'   -> v_start_date: {v_start_date}')

curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
print(f'\nData as of {curr_datetime}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, TM_KEY_WK, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME --, AREA_TYPE, AREA_CD, AREA_NAME
        , MAX(ACTUAL_AS_OF) ACTUAL_AS_OF
        , SUM(CASE WHEN AREA_TYPE = 'C' THEN ACTUAL_SNAP END) C
        , SUM(CASE WHEN AREA_TYPE = 'P' THEN ACTUAL_SNAP END) P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN ACTUAL_SNAP END) G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN ACTUAL_SNAP END) H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN ACTUAL_SNAP END) HH
        , MAX(PPN_TM) PPN_TM
    FROM GEOSPCAPPO.AGG_PERF_NEWCO
    WHERE METRIC_CD IN (
        'B0R000100GEO' --Total Revenue (Geo)
        , 'B1R000100' --Prepaid Revenue
        , 'B2R010100' --Postpaid Revenue B2C
        , 'TB3R000100' --TOL Revenue
        , 'TB4R000100' --TVS Revenue
         
        , 'B1S000600' --Prepaid Active Caller 30D Rolling
        , 'B2S010600' --Postpaid Reported SubBase B2C
        , 'TB3S000600' --FTTx Reported SubBase
        , 'TB4S000500' --TVS Active Subs
         
        , 'B1S000101CS' --Prepaid Gross Adds
        , 'B2S010100CS' --Postpaid Gross Adds B2C
        , 'TB3S000102CS' --TOL Gross Adds Connected : Consumer
        , 'TB4S001400CS' --TVS Now Gross Adds
         
        , 'B1S000200' --Prepaid Churn Subs
        , 'B2S010200' --Postpaid Churn Subs B2C
        , 'B2S010201' --Postpaid Churn Subs Voluntary B2C
        , 'B2S010202' --Postpaid Churn Subs Involuntary B2C
        , 'TSER13100' --FTTx Churn Subs
        , 'TSER13102' --FTTx Churn Subs Voluntary
        , 'TSER13103' --FTTx Churn Subs Involuntary
        , 'TSER13104' --FTTx Churn Subs Involuntary (Collection Forecast)
        , 'TSER14100' --TVS Churn Subs
        , 'TSER14102' --TVS Churn Subs Voluntary
        , 'TSER14103' --TVS Churn Subs Involuntary
        )
    AND AREA_TYPE IN ('C','P','G','H','HH')
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, TM_KEY_WK, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME
""")


try:
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

    print(f'\nDataFrame: {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Parameter input...
   -> v_start_date: 20240101

Data as of 2025-06-20, 09:49:08

DataFrame: 12860 rows, 16 columns


In [3]:
''' Automate Currently Period '''

curr_yr = chk_src_df['TM_KEY_YR'].max()
prev_yr = chk_src_df['TM_KEY_YR'].drop_duplicates().sort_values().shift().max().astype(int)

curr_qtr = chk_src_df['TM_KEY_QTR'].max()
prev_qtr = chk_src_df['TM_KEY_QTR'].drop_duplicates().sort_values().shift().max().astype(int)

curr_mth = chk_src_df['TM_KEY_MTH'].max()
prev_mth = chk_src_df['TM_KEY_MTH'].drop_duplicates().sort_values().shift().max().astype(int)

curr_wk = chk_src_df['TM_KEY_WK'].max()
prev_wk = chk_src_df['TM_KEY_WK'].drop_duplicates().sort_values().shift().max().astype(int)
last_3_wk = chk_src_df['TM_KEY_WK'].drop_duplicates().sort_values().shift(3).max().astype(int)

# curr_day = chk_src_df.loc[chk_src_df['ACTUAL_SNAP'] > 0]['ACTUAL_AS_OF'].max().astype(int)
# prev_day = chk_src_df.loc[chk_src_df['ACTUAL_SNAP'] > 0]['ACTUAL_AS_OF'].drop_duplicates().shift().max().astype(int)

In [4]:
# chk_src_df.tail()

## Period Parameter

In [5]:
''' Parameter '''

# v_tm_key_qtr = curr_qtr
v_tm_key_qtr = 20241

v_tm_key_mth = curr_mth
# v_tm_key_mth = prev_mth
# v_tm_key_mth = 202505

# v_tm_key_wk = curr_wk
v_tm_key_wk = last_3_wk
# v_tm_key_wk = 2025020

print(f'v_tm_key_qtr: {v_tm_key_qtr}')
print(f'v_tm_key_mth: {v_tm_key_mth}')
print(f'v_tm_key_wk: {v_tm_key_wk}')

v_tm_key_qtr: 20241
v_tm_key_mth: 202506
v_tm_key_wk: 2025022


## Revenue(Geo) by Period

### Yearly

In [6]:
''' Revenue(Geo) Yearly '''

v_metric_list = [
    'B0R000100GEO' #Total Revenue (Geo)
    , 'B1R000100' #Prepaid Revenue
    , 'B2R010100' #Postpaid Revenue B2C
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]

rev_geo_yearly_df = chk_src_df[['TM_KEY_YR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
rev_geo_yearly_df = rev_geo_yearly_df.loc[rev_geo_yearly_df['METRIC_CD'].isin(v_metric_list)]

rev_geo_yearly_df['TOTAL(Geo)'] = np.where(rev_geo_yearly_df['METRIC_CD']=='B0R000100GEO', rev_geo_yearly_df['P'], 0)
rev_geo_yearly_df['PRE'] = np.where(rev_geo_yearly_df['METRIC_CD']=='B1R000100', rev_geo_yearly_df['P'], 0)
rev_geo_yearly_df['POST_B2C'] = np.where(rev_geo_yearly_df['METRIC_CD']=='B2R010100', rev_geo_yearly_df['P'], 0)
rev_geo_yearly_df['TOL'] = np.where(rev_geo_yearly_df['METRIC_CD']=='TB3R000100', rev_geo_yearly_df['P'], 0)
rev_geo_yearly_df['TVS'] = np.where(rev_geo_yearly_df['METRIC_CD']=='TB4R000100', rev_geo_yearly_df['P'], 0)

rev_geo_yearly_df = rev_geo_yearly_df.groupby('TM_KEY_YR').agg({'PPN_TM':'max', 'TOTAL(Geo)':'sum', 'PRE':'sum', 'POST_B2C':'sum', 'TOL':'sum', 'TVS':'sum'})
rev_geo_yearly_df = rev_geo_yearly_df.fillna(0).sort_values(by=['TM_KEY_YR']).reset_index()
rev_geo_yearly_df = rev_geo_yearly_df[['TM_KEY_YR', 'PPN_TM', 'TOTAL(Geo)', 'PRE', 'POST_B2C', 'TOL', 'TVS']]

mod_col_list = rev_geo_yearly_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    rev_geo_yearly_df[col] = rev_geo_yearly_df[col].apply(lambda x: format(x, ',.0f'))
rev_geo_yearly_df

,TM_KEY_YR,PPN_TM,TOTAL(Geo),PRE,POST_B2C,TOL,TVS
0,2024,2025-06-20 00:48:43,"142,125,302,342","48,319,930,436","70,959,878,334","18,827,249,425","4,018,244,147"
1,2025,2025-06-20 00:48:43,"66,152,429,639","22,957,236,125","32,469,533,683","8,971,381,070","1,585,830,232"


### Quarterly

In [7]:
''' Revenue(Geo) Quarterly '''

v_metric_list = [
    'B0R000100GEO' #Total Revenue (Geo)
    , 'B1R000100' #Prepaid Revenue
    , 'B2R010100' #Postpaid Revenue B2C
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]

rev_geo_quarterly_df = chk_src_df[['TM_KEY_QTR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
rev_geo_quarterly_df = rev_geo_quarterly_df.loc[rev_geo_quarterly_df['METRIC_CD'].isin(v_metric_list)]
rev_geo_quarterly_df = rev_geo_quarterly_df.loc[rev_geo_quarterly_df['TM_KEY_QTR']>=v_tm_key_qtr]

rev_geo_quarterly_df['TOTAL(Geo)'] = np.where(rev_geo_quarterly_df['METRIC_CD']=='B0R000100GEO', rev_geo_quarterly_df['P'], 0)
rev_geo_quarterly_df['PRE'] = np.where(rev_geo_quarterly_df['METRIC_CD']=='B1R000100', rev_geo_quarterly_df['P'], 0)
rev_geo_quarterly_df['POST_B2C'] = np.where(rev_geo_quarterly_df['METRIC_CD']=='B2R010100', rev_geo_quarterly_df['P'], 0)
rev_geo_quarterly_df['TOL'] = np.where(rev_geo_quarterly_df['METRIC_CD']=='TB3R000100', rev_geo_quarterly_df['P'], 0)
rev_geo_quarterly_df['TVS'] = np.where(rev_geo_quarterly_df['METRIC_CD']=='TB4R000100', rev_geo_quarterly_df['P'], 0)

rev_geo_quarterly_df = rev_geo_quarterly_df.groupby('TM_KEY_QTR').agg({'PPN_TM':'max', 'TOTAL(Geo)':'sum', 'PRE':'sum', 'POST_B2C':'sum', 'TOL':'sum', 'TVS':'sum'})
rev_geo_quarterly_df = rev_geo_quarterly_df.fillna(0).sort_values(by=['TM_KEY_QTR']).reset_index()
rev_geo_quarterly_df = rev_geo_quarterly_df[['TM_KEY_QTR', 'PPN_TM', 'TOTAL(Geo)', 'PRE', 'POST_B2C', 'TOL', 'TVS']]

mod_col_list = rev_geo_quarterly_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    rev_geo_quarterly_df[col] = rev_geo_quarterly_df[col].apply(lambda x: format(x, ',.0f'))
rev_geo_quarterly_df

,TM_KEY_QTR,PPN_TM,TOTAL(Geo),PRE,POST_B2C,TOL,TVS
0,20241,2025-06-20 00:48:43,"35,361,933,475","11,842,176,786","17,936,624,018","4,569,920,004","1,013,212,668"
1,20242,2025-06-20 00:48:43,"35,284,532,532","11,858,030,293","17,792,137,434","4,665,781,597","968,583,208"
2,20243,2025-06-20 00:48:43,"35,706,816,037","12,164,985,004","17,628,033,120","4,803,785,483","1,110,012,430"
3,20244,2025-06-20 00:48:43,"35,772,020,298","12,454,738,354","17,603,083,763","4,787,762,341","926,435,841"
4,20251,2025-06-20 00:48:43,"35,649,443,458","12,375,356,967","17,580,761,024","4,810,187,607","883,137,861"
5,20252,2025-06-20 00:48:43,"30,502,986,181","10,581,879,158","14,888,772,658","4,161,193,463","702,692,372"


### Monthly

In [8]:
''' Revenue(Geo) Monthly '''

v_metric_list = [
    'B0R000100GEO' #Total Revenue (Geo)
    , 'B1R000100' #Prepaid Revenue
    , 'B2R010100' #Postpaid Revenue B2C
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]

rev_geo_monthly_df = chk_src_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
rev_geo_monthly_df = rev_geo_monthly_df.loc[rev_geo_monthly_df['METRIC_CD'].isin(v_metric_list)]
rev_geo_monthly_df = rev_geo_monthly_df.loc[rev_geo_monthly_df['TM_KEY_MTH']>=v_tm_key_mth]

rev_geo_monthly_df['TOTAL(Geo)'] = np.where(rev_geo_monthly_df['METRIC_CD']=='B0R000100GEO', rev_geo_monthly_df['P'], 0)
rev_geo_monthly_df['PRE'] = np.where(rev_geo_monthly_df['METRIC_CD']=='B1R000100', rev_geo_monthly_df['P'], 0)
rev_geo_monthly_df['POST_B2C'] = np.where(rev_geo_monthly_df['METRIC_CD']=='B2R010100', rev_geo_monthly_df['P'], 0)
rev_geo_monthly_df['TOL'] = np.where(rev_geo_monthly_df['METRIC_CD']=='TB3R000100', rev_geo_monthly_df['P'], 0)
rev_geo_monthly_df['TVS'] = np.where(rev_geo_monthly_df['METRIC_CD']=='TB4R000100', rev_geo_monthly_df['P'], 0)

rev_geo_monthly_df = rev_geo_monthly_df.groupby('TM_KEY_MTH').agg({'PPN_TM':'max', 'TOTAL(Geo)':'sum', 'PRE':'sum', 'POST_B2C':'sum', 'TOL':'sum', 'TVS':'sum'})
rev_geo_monthly_df = rev_geo_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
rev_geo_monthly_df = rev_geo_monthly_df[['TM_KEY_MTH', 'PPN_TM', 'TOTAL(Geo)', 'PRE', 'POST_B2C', 'TOL', 'TVS']]

mod_col_list = rev_geo_monthly_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    rev_geo_monthly_df[col] = rev_geo_monthly_df[col].apply(lambda x: format(x, ',.0f'))
rev_geo_monthly_df

,TM_KEY_MTH,PPN_TM,TOTAL(Geo),PRE,POST_B2C,TOL,TVS
0,202506,2025-06-20 00:48:43,"6,592,930,383","2,248,503,693","3,134,009,496","934,240,883","107,727,781"


### Weekly

In [9]:
''' Revenue(Geo) Weekly '''

v_metric_list = [
    'B0R000100GEO' #Total Revenue (Geo)
    , 'B1R000100' #Prepaid Revenue
    , 'B2R010100' #Postpaid Revenue B2C
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]

rev_geo_weekly_df = chk_src_df[['TM_KEY_WK', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
rev_geo_weekly_df = rev_geo_weekly_df.loc[rev_geo_weekly_df['METRIC_CD'].isin(v_metric_list)]
# rev_geo_weekly_df = rev_geo_weekly_df.loc[rev_geo_weekly_df['TM_KEY_WK']>=v_tm_key_wk]

rev_geo_weekly_df['TOTAL(Geo)'] = np.where(rev_geo_weekly_df['METRIC_CD']=='B0R000100GEO', rev_geo_weekly_df['P'], 0)
rev_geo_weekly_df['PRE'] = np.where(rev_geo_weekly_df['METRIC_CD']=='B1R000100', rev_geo_weekly_df['P'], 0)
rev_geo_weekly_df['POST_B2C'] = np.where(rev_geo_weekly_df['METRIC_CD']=='B2R010100', rev_geo_weekly_df['P'], 0)
rev_geo_weekly_df['TOL'] = np.where(rev_geo_weekly_df['METRIC_CD']=='TB3R000100', rev_geo_weekly_df['P'], 0)
rev_geo_weekly_df['TVS'] = np.where(rev_geo_weekly_df['METRIC_CD']=='TB4R000100', rev_geo_weekly_df['P'], 0)

rev_geo_weekly_df = rev_geo_weekly_df.groupby('TM_KEY_WK').agg({'TM_KEY_DAY':'max', 'PPN_TM':'max', 'TOTAL(Geo)':'sum', 'PRE':'sum', 'POST_B2C':'sum', 'TOL':'sum', 'TVS':'sum'})
rev_geo_weekly_df = rev_geo_weekly_df.fillna(0).sort_values(by=['TM_KEY_WK']).reset_index()
rev_geo_weekly_df = rev_geo_weekly_df[['TM_KEY_WK', 'TM_KEY_DAY', 'PPN_TM', 'TOTAL(Geo)', 'PRE', 'POST_B2C', 'TOL', 'TVS']]
rev_geo_weekly_df.rename(columns={'TM_KEY_DAY':'AS_DAY'}, inplace=True)

mod_col_list = rev_geo_weekly_df.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    rev_geo_weekly_df[col] = rev_geo_weekly_df[col].apply(lambda x: format(x, ',.0f'))
rev_geo_weekly_df

,TM_KEY_WK,AS_DAY,PPN_TM,TOTAL(Geo),PRE,POST_B2C,TOL,TVS
0,2025022,20250601,2025-06-20 00:48:43,"169,983,041","165,152,372","4,830,669",0,0
1,2025023,20250608,2025-06-20 00:48:43,"2,756,559,245","993,554,639","1,356,302,562","365,933,686","40,768,358"
2,2025024,20250615,2025-06-20 00:48:43,"2,694,250,949","871,818,390","1,373,430,974","402,469,675","46,531,909"
3,2025025,20250619,2025-06-20 00:48:43,"972,137,148","217,978,292","399,445,291","165,837,522","20,427,514"


### Daily
    The information is valid until 6 June 2025

In [10]:
''' Revenue(Geo) Daily '''

v_metric_list = [
    'B0R000100GEO' #Total Revenue (Geo)
    , 'B1R000100' #Prepaid Revenue
    , 'B2R010100' #Postpaid Revenue B2C
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]

rev_geo_daily_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
rev_geo_daily_df = rev_geo_daily_df.loc[rev_geo_daily_df['METRIC_CD'].isin(v_metric_list)]
rev_geo_daily_df = rev_geo_daily_df.loc[rev_geo_daily_df['TM_KEY_MTH']>=v_tm_key_mth]

rev_geo_daily_df['TOTAL(Geo)'] = np.where(rev_geo_daily_df['METRIC_CD']=='B0R000100GEO', rev_geo_daily_df['P'], 0)
rev_geo_daily_df['PRE'] = np.where(rev_geo_daily_df['METRIC_CD']=='B1R000100', rev_geo_daily_df['P'], 0)
rev_geo_daily_df['POST_B2C'] = np.where(rev_geo_daily_df['METRIC_CD']=='B2R010100', rev_geo_daily_df['P'], 0)
rev_geo_daily_df['TOL'] = np.where(rev_geo_daily_df['METRIC_CD']=='TB3R000100', rev_geo_daily_df['P'], 0)
rev_geo_daily_df['TVS'] = np.where(rev_geo_daily_df['METRIC_CD']=='TB4R000100', rev_geo_daily_df['P'], 0)

rev_geo_daily_df = rev_geo_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max', 'TOTAL(Geo)':'sum', 'PRE':'sum', 'POST_B2C':'sum', 'TOL':'sum', 'TVS':'sum'})
rev_geo_daily_df = rev_geo_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY'], ascending=False).reset_index()
rev_geo_daily_df = rev_geo_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'TOTAL(Geo)', 'PRE', 'POST_B2C', 'TOL', 'TVS']]

mod_col_list = rev_geo_daily_df.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    rev_geo_daily_df[col] = rev_geo_daily_df[col].apply(lambda x: format(x, ',.0f'))
rev_geo_daily_df

,TM_KEY_MTH,TM_KEY_DAY,PPN_TM,TOTAL(Geo),PRE,POST_B2C,TOL,TVS
0,202506,20250619,2025-06-20 00:48:43,0,0,0,0,0
1,202506,20250618,2025-06-20 00:48:43,"91,545,237",0,0,0,0
2,202506,20250617,2025-06-20 00:48:43,"185,465,535","108,562,243",0,0,0
3,202506,20250616,2025-06-20 00:48:43,"695,126,376","109,416,049","399,445,291","165,837,522","20,427,514"
4,202506,20250615,2025-06-20 00:48:43,"175,456,294","99,806,342","75,649,952",0,0
5,202506,20250614,2025-06-20 00:48:43,"198,030,050","122,134,305","75,895,745",0,0
6,202506,20250613,2025-06-20 00:48:43,"772,854,506","122,393,212","460,258,458","170,379,045","19,823,791"
7,202506,20250612,2025-06-20 00:48:43,"205,665,212","125,111,134","80,554,078",0,0
8,202506,20250611,2025-06-20 00:48:43,"213,214,372","131,249,479","81,964,893",0,0
9,202506,20250610,2025-06-20 00:48:43,"915,273,505","140,558,699","515,916,057","232,090,630","26,708,119"


## Subscriber by Period

### Yearly

In [11]:
''' Subscriber Yearly '''

v_metric_list = [
    'B1S000600' #Prepaid Active Caller 30D Rolling
    , 'B2S010600' #Postpaid Reported SubBase B2C
    , 'TB3S000600' #FTTx Reported SubBase
    , 'TB4S000500' #TVS Active Subs
    ]

sub_yearly_df = chk_src_df.loc[chk_src_df['TM_KEY_DAY']==chk_src_df.groupby(['TM_KEY_YR', 'METRIC_CD'])['ACTUAL_AS_OF'].transform('max')].copy() # ACTUAL_AGG_YR

sub_yearly_df = sub_yearly_df[['TM_KEY_YR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
sub_yearly_df = sub_yearly_df.loc[sub_yearly_df['METRIC_CD'].isin(v_metric_list)]

sub_yearly_df['PRE_CALL_30D'] = np.where(sub_yearly_df['METRIC_CD']=='B1S000600', sub_yearly_df['P'], 0)
sub_yearly_df['POST_RPT_B2C'] = np.where(sub_yearly_df['METRIC_CD']=='B2S010600', sub_yearly_df['P'], 0)
sub_yearly_df['FTTX_RPT'] = np.where(sub_yearly_df['METRIC_CD']=='TB3S000600', sub_yearly_df['P'], 0)
sub_yearly_df['TVS_ACTV'] = np.where(sub_yearly_df['METRIC_CD']=='TB4S000500', sub_yearly_df['P'], 0)

sub_yearly_df = sub_yearly_df.groupby('TM_KEY_YR').agg({'PPN_TM':'max', 'PRE_CALL_30D':'sum', 'POST_RPT_B2C':'sum', 'FTTX_RPT':'sum', 'TVS_ACTV':'sum'})
sub_yearly_df = sub_yearly_df.fillna(0).sort_values(by=['TM_KEY_YR']).reset_index()
sub_yearly_df = sub_yearly_df[['TM_KEY_YR', 'PPN_TM', 'PRE_CALL_30D', 'POST_RPT_B2C', 'FTTX_RPT', 'TVS_ACTV']]

mod_col_list = sub_yearly_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    sub_yearly_df[col] = sub_yearly_df[col].apply(lambda x: format(x, ',.0f'))
sub_yearly_df

,TM_KEY_YR,PPN_TM,PRE_CALL_30D,POST_RPT_B2C,FTTX_RPT,TVS_ACTV
0,2024,2025-06-20 00:48:43,"28,100,141","14,158,698","3,154,889","1,180,410"
1,2025,2025-06-20 00:48:43,"26,296,230","14,052,647","3,205,591","1,119,316"


### Quarterly

In [12]:
''' Subscriber Quarterly '''

v_metric_list = [
    'B1S000600' #Prepaid Active Caller 30D Rolling
    , 'B2S010600' #Postpaid Reported SubBase B2C
    , 'TB3S000600' #FTTx Reported SubBase
    , 'TB4S000500' #TVS Active Subs
    ]

sub_quarterly_df = chk_src_df.loc[chk_src_df['TM_KEY_DAY']==chk_src_df.groupby(['TM_KEY_QTR', 'METRIC_CD'])['ACTUAL_AS_OF'].transform('max')].copy() # ACTUAL_AGG_YR

sub_quarterly_df = sub_quarterly_df[['TM_KEY_QTR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
sub_quarterly_df = sub_quarterly_df.loc[sub_quarterly_df['METRIC_CD'].isin(v_metric_list)]
sub_quarterly_df = sub_quarterly_df.loc[sub_quarterly_df['TM_KEY_QTR']>=v_tm_key_qtr]

sub_quarterly_df['PRE_CALL_30D'] = np.where(sub_quarterly_df['METRIC_CD']=='B1S000600', sub_quarterly_df['P'], 0)
sub_quarterly_df['POST_RPT_B2C'] = np.where(sub_quarterly_df['METRIC_CD']=='B2S010600', sub_quarterly_df['P'], 0)
sub_quarterly_df['FTTX_RPT'] = np.where(sub_quarterly_df['METRIC_CD']=='TB3S000600', sub_quarterly_df['P'], 0)
sub_quarterly_df['TVS_ACTV'] = np.where(sub_quarterly_df['METRIC_CD']=='TB4S000500', sub_quarterly_df['P'], 0)

sub_quarterly_df = sub_quarterly_df.groupby('TM_KEY_QTR').agg({'PPN_TM':'max', 'PRE_CALL_30D':'sum', 'POST_RPT_B2C':'sum', 'FTTX_RPT':'sum', 'TVS_ACTV':'sum'})
sub_quarterly_df = sub_quarterly_df.fillna(0).sort_values(by=['TM_KEY_QTR']).reset_index()
sub_quarterly_df = sub_quarterly_df[['TM_KEY_QTR', 'PPN_TM', 'PRE_CALL_30D', 'POST_RPT_B2C', 'FTTX_RPT', 'TVS_ACTV']]

mod_col_list = sub_quarterly_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    sub_quarterly_df[col] = sub_quarterly_df[col].apply(lambda x: format(x, ',.0f'))
sub_quarterly_df

,TM_KEY_QTR,PPN_TM,PRE_CALL_30D,POST_RPT_B2C,FTTX_RPT,TVS_ACTV
0,20241,2025-06-20 00:48:43,0,0,"3,090,287","1,258,065"
1,20242,2025-06-20 00:48:43,"27,800,517","14,226,167","3,107,084","1,236,606"
2,20243,2025-06-20 00:48:43,"27,505,003","14,186,615","3,127,304","1,205,116"
3,20244,2025-06-20 00:48:43,"28,100,141","14,158,698","3,154,889","1,180,410"
4,20251,2025-06-20 00:48:43,"27,140,083","14,084,812","3,187,717","1,153,137"
5,20252,2025-06-20 00:48:43,"26,296,230","14,052,647","3,205,591","1,119,316"


### Monthly

In [13]:
''' Subscriber Monthly '''

v_metric_list = [
    'B1S000600' #Prepaid Active Caller 30D Rolling
    , 'B2S010600' #Postpaid Reported SubBase B2C
    , 'TB3S000600' #FTTx Reported SubBase
    , 'TB4S000500' #TVS Active Subs
    ]

sub_monthly_df = chk_src_df.loc[chk_src_df['TM_KEY_DAY']==chk_src_df.groupby(['TM_KEY_MTH', 'METRIC_CD'])['ACTUAL_AS_OF'].transform('max')].copy() # ACTUAL_AGG_YR

sub_monthly_df = sub_monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
sub_monthly_df = sub_monthly_df.loc[sub_monthly_df['METRIC_CD'].isin(v_metric_list)]
sub_monthly_df = sub_monthly_df.loc[sub_monthly_df['TM_KEY_MTH']>=v_tm_key_mth]

sub_monthly_df['PRE_CALL_30D'] = np.where(sub_monthly_df['METRIC_CD']=='B1S000600', sub_monthly_df['P'], 0)
sub_monthly_df['POST_RPT_B2C'] = np.where(sub_monthly_df['METRIC_CD']=='B2S010600', sub_monthly_df['P'], 0)
sub_monthly_df['FTTX_RPT'] = np.where(sub_monthly_df['METRIC_CD']=='TB3S000600', sub_monthly_df['P'], 0)
sub_monthly_df['TVS_ACTV'] = np.where(sub_monthly_df['METRIC_CD']=='TB4S000500', sub_monthly_df['P'], 0)

sub_monthly_df = sub_monthly_df.groupby('TM_KEY_MTH').agg({'PPN_TM':'max', 'PRE_CALL_30D':'sum', 'POST_RPT_B2C':'sum', 'FTTX_RPT':'sum', 'TVS_ACTV':'sum'})
sub_monthly_df = sub_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
sub_monthly_df = sub_monthly_df[['TM_KEY_MTH', 'PPN_TM', 'PRE_CALL_30D', 'POST_RPT_B2C', 'FTTX_RPT', 'TVS_ACTV']]

mod_col_list = sub_monthly_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    sub_monthly_df[col] = sub_monthly_df[col].apply(lambda x: format(x, ',.0f'))
sub_monthly_df

,TM_KEY_MTH,PPN_TM,PRE_CALL_30D,POST_RPT_B2C,FTTX_RPT,TVS_ACTV
0,202506,2025-06-20 00:48:43,"26,296,230","14,052,647","3,205,591","1,119,316"


### Weekly

In [14]:
''' Subscriber Weekly '''

v_metric_list = [
    'B1S000600' #Prepaid Active Caller 30D Rolling
    , 'B2S010600' #Postpaid Reported SubBase B2C
    , 'TB3S000600' #FTTx Reported SubBase
    , 'TB4S000500' #TVS Active Subs
    ]

sub_weekly_df = chk_src_df.loc[chk_src_df['TM_KEY_DAY']==chk_src_df.groupby(['TM_KEY_WK', 'METRIC_CD'])['ACTUAL_AS_OF'].transform('max')].copy() # ACTUAL_AGG_YR

sub_weekly_df = sub_weekly_df[['TM_KEY_WK', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
sub_weekly_df = sub_weekly_df.loc[sub_weekly_df['METRIC_CD'].isin(v_metric_list)]
# sub_weekly_df = sub_weekly_df.loc[sub_weekly_df['TM_KEY_WK']>=v_tm_key_wk]

sub_weekly_df['PRE_CALL_30D'] = np.where(sub_weekly_df['METRIC_CD']=='B1S000600', sub_weekly_df['P'], 0)
sub_weekly_df['POST_RPT_B2C'] = np.where(sub_weekly_df['METRIC_CD']=='B2S010600', sub_weekly_df['P'], 0)
sub_weekly_df['FTTX_RPT'] = np.where(sub_weekly_df['METRIC_CD']=='TB3S000600', sub_weekly_df['P'], 0)
sub_weekly_df['TVS_ACTV'] = np.where(sub_weekly_df['METRIC_CD']=='TB4S000500', sub_weekly_df['P'], 0)

sub_weekly_df = sub_weekly_df.groupby('TM_KEY_WK').agg({'TM_KEY_DAY':'max', 'PPN_TM':'max', 'PRE_CALL_30D':'sum', 'POST_RPT_B2C':'sum', 'FTTX_RPT':'sum', 'TVS_ACTV':'sum'})
sub_weekly_df = sub_weekly_df.fillna(0).sort_values(by=['TM_KEY_WK']).reset_index()
sub_weekly_df = sub_weekly_df[['TM_KEY_WK', 'TM_KEY_DAY', 'PPN_TM', 'PRE_CALL_30D', 'POST_RPT_B2C', 'FTTX_RPT', 'TVS_ACTV']]
sub_weekly_df.rename(columns={'TM_KEY_DAY':'AS_DAY'}, inplace=True)

mod_col_list = sub_weekly_df.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    sub_weekly_df[col] = sub_weekly_df[col].apply(lambda x: format(x, ',.0f'))
sub_weekly_df

,TM_KEY_WK,AS_DAY,PPN_TM,PRE_CALL_30D,POST_RPT_B2C,FTTX_RPT,TVS_ACTV
0,2025022,20250601,2025-06-20 00:48:43,"26,438,734","14,073,719","3,205,472","1,128,753"
1,2025023,20250608,2025-06-20 00:48:43,"26,362,107","14,064,757","3,202,722","1,126,877"
2,2025024,20250615,2025-06-20 00:48:43,"26,318,308","14,045,663","3,201,297","1,119,850"
3,2025025,20250618,2025-06-20 00:48:43,"26,296,230","14,052,647","3,205,591","1,119,316"


### Daily

In [15]:
''' Subscriber Daily '''

v_metric_list = [
    'B1S000600' #Prepaid Active Caller 30D Rolling
    , 'B2S010600' #Postpaid Reported SubBase B2C
    , 'TB3S000600' #FTTx Reported SubBase
    , 'TB4S000500' #TVS Active Subs
    ]

sub_daily_df = chk_src_df.loc[chk_src_df['TM_KEY_DAY']==chk_src_df.groupby(['TM_KEY_WK', 'METRIC_CD'])['ACTUAL_AS_OF'].transform('max')].copy() # ACTUAL_AGG_YR

sub_daily_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
sub_daily_df = sub_daily_df.loc[sub_daily_df['METRIC_CD'].isin(v_metric_list)]
sub_daily_df = sub_daily_df.loc[sub_daily_df['TM_KEY_MTH']>=v_tm_key_mth]

sub_daily_df['PRE_CALL_30D'] = np.where(sub_daily_df['METRIC_CD']=='B1S000600', sub_daily_df['P'], 0)
sub_daily_df['POST_RPT_B2C'] = np.where(sub_daily_df['METRIC_CD']=='B2S010600', sub_daily_df['P'], 0)
sub_daily_df['FTTX_RPT'] = np.where(sub_daily_df['METRIC_CD']=='TB3S000600', sub_daily_df['P'], 0)
sub_daily_df['TVS_ACTV'] = np.where(sub_daily_df['METRIC_CD']=='TB4S000500', sub_daily_df['P'], 0)

sub_daily_df = sub_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max', 'PRE_CALL_30D':'sum', 'POST_RPT_B2C':'sum', 'FTTX_RPT':'sum', 'TVS_ACTV':'sum'})
sub_daily_df = sub_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY'], ascending=False).reset_index()
sub_daily_df = sub_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'PRE_CALL_30D', 'POST_RPT_B2C', 'FTTX_RPT', 'TVS_ACTV']]

mod_col_list = sub_daily_df.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    sub_daily_df[col] = sub_daily_df[col].apply(lambda x: format(x, ',.0f'))
sub_daily_df

,TM_KEY_MTH,TM_KEY_DAY,PPN_TM,PRE_CALL_30D,POST_RPT_B2C,FTTX_RPT,TVS_ACTV
0,202506,20250619,2025-06-20 00:48:43,0,0,0,0
1,202506,20250618,2025-06-20 00:48:43,"26,296,230",0,0,"1,119,316"
2,202506,20250617,2025-06-20 00:48:43,"26,312,653","14,052,647","3,205,591","1,119,568"
3,202506,20250616,2025-06-20 00:48:43,"9,847,880","14,049,707","3,203,535","1,119,850"
4,202506,20250615,2025-06-20 00:48:43,"26,318,308","14,045,663","3,201,297","1,119,850"
5,202506,20250614,2025-06-20 00:48:43,"26,334,422","14,052,198","3,204,933","1,119,850"
6,202506,20250613,2025-06-20 00:48:43,"26,350,345","14,054,991","3,202,631","1,119,850"
7,202506,20250612,2025-06-20 00:48:43,"26,328,436","14,149,857","3,200,418","1,119,850"
8,202506,20250611,2025-06-20 00:48:43,"26,342,832","14,147,431","3,203,476","1,126,877"
9,202506,20250610,2025-06-20 00:48:43,"26,377,226","14,052,954","3,201,325","1,126,877"


## Gross Adds by Period

### Yearly

In [16]:
''' Gross Adds Yearly '''

v_metric_list = [
    'B1S000101CS' #Prepaid Gross Adds
    , 'B2S010100CS' #Postpaid Gross Adds B2C
    , 'TB3S000102CS' #TOL Gross Adds Connected : Consumer
    , 'TB4S001400CS' #TVS Now Gross Adds
    ]

ga_yearly_df = chk_src_df[['TM_KEY_YR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
ga_yearly_df = ga_yearly_df.loc[ga_yearly_df['METRIC_CD'].isin(v_metric_list)]

ga_yearly_df['PRE_GA'] = np.where(ga_yearly_df['METRIC_CD']=='B1S000101CS', ga_yearly_df['P'], 0)
ga_yearly_df['POST_GA_B2C'] = np.where(ga_yearly_df['METRIC_CD']=='B2S010100CS', ga_yearly_df['P'], 0)
ga_yearly_df['TOL_GA_CONN_CON'] = np.where(ga_yearly_df['METRIC_CD']=='TB3S000102CS', ga_yearly_df['P'], 0)
ga_yearly_df['TVS_NOW_GA'] = np.where(ga_yearly_df['METRIC_CD']=='TB4S001400CS', ga_yearly_df['P'], 0)

ga_yearly_df = ga_yearly_df.groupby('TM_KEY_YR').agg({'PPN_TM':'max', 'PRE_GA':'sum', 'POST_GA_B2C':'sum', 'TOL_GA_CONN_CON':'sum', 'TVS_NOW_GA':'sum'})
ga_yearly_df = ga_yearly_df.fillna(0).sort_values(by=['TM_KEY_YR']).reset_index()
ga_yearly_df = ga_yearly_df[['TM_KEY_YR', 'PPN_TM', 'PRE_GA', 'POST_GA_B2C', 'TOL_GA_CONN_CON', 'TVS_NOW_GA']]

mod_col_list = ga_yearly_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    ga_yearly_df[col] = ga_yearly_df[col].apply(lambda x: format(x, ',.0f'))
ga_yearly_df

,TM_KEY_YR,PPN_TM,PRE_GA,POST_GA_B2C,TOL_GA_CONN_CON,TVS_NOW_GA
0,2024,2025-06-20 00:48:43,"30,607,404","2,642,885","558,527","7,682"
1,2025,2025-06-20 00:48:43,"11,444,273","1,055,799","278,363","9,007"


### Quarterly

In [17]:
''' Gross Adds Quarterly '''

v_metric_list = [
    'B1S000101CS' #Prepaid Gross Adds
    , 'B2S010100CS' #Postpaid Gross Adds B2C
    , 'TB3S000102CS' #TOL Gross Adds Connected : Consumer
    , 'TB4S001400CS' #TVS Now Gross Adds
    ]

ga_quarterly_df = chk_src_df[['TM_KEY_QTR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
ga_quarterly_df = ga_quarterly_df.loc[ga_quarterly_df['METRIC_CD'].isin(v_metric_list)]
ga_quarterly_df = ga_quarterly_df.loc[ga_quarterly_df['TM_KEY_QTR']>=v_tm_key_qtr]

ga_quarterly_df['PRE_GA'] = np.where(ga_quarterly_df['METRIC_CD']=='B1S000101CS', ga_quarterly_df['P'], 0)
ga_quarterly_df['POST_GA_B2C'] = np.where(ga_quarterly_df['METRIC_CD']=='B2S010100CS', ga_quarterly_df['P'], 0)
ga_quarterly_df['TOL_GA_CONN_CON'] = np.where(ga_quarterly_df['METRIC_CD']=='TB3S000102CS', ga_quarterly_df['P'], 0)
ga_quarterly_df['TVS_NOW_GA'] = np.where(ga_quarterly_df['METRIC_CD']=='TB4S001400CS', ga_quarterly_df['P'], 0)

ga_quarterly_df = ga_quarterly_df.groupby('TM_KEY_QTR').agg({'PPN_TM':'max', 'PRE_GA':'sum', 'POST_GA_B2C':'sum', 'TOL_GA_CONN_CON':'sum', 'TVS_NOW_GA':'sum'})
ga_quarterly_df = ga_quarterly_df.fillna(0).sort_values(by=['TM_KEY_QTR']).reset_index()
ga_quarterly_df = ga_quarterly_df[['TM_KEY_QTR', 'PPN_TM', 'PRE_GA', 'POST_GA_B2C', 'TOL_GA_CONN_CON', 'TVS_NOW_GA']]

mod_col_list = ga_quarterly_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    ga_quarterly_df[col] = ga_quarterly_df[col].apply(lambda x: format(x, ',.0f'))
ga_quarterly_df

,TM_KEY_QTR,PPN_TM,PRE_GA,POST_GA_B2C,TOL_GA_CONN_CON,TVS_NOW_GA
0,20241,2025-06-20 00:48:43,"8,953,743","701,773","130,958",0
1,20242,2025-06-20 00:48:43,"7,514,316","698,111","141,474",0
2,20243,2025-06-20 00:48:43,"7,156,127","662,059","147,285","4,992"
3,20244,2025-06-20 00:48:43,"6,983,218","580,942","138,810","2,690"
4,20251,2025-06-20 00:48:43,"6,673,218","569,340","145,692","3,337"
5,20252,2025-06-20 00:48:43,"4,771,055","486,459","132,671","5,670"


### Monthly

In [18]:
''' Gross Adds Monthly '''

v_metric_list = [
    'B1S000101CS' #Prepaid Gross Adds
    , 'B2S010100CS' #Postpaid Gross Adds B2C
    , 'TB3S000102CS' #TOL Gross Adds Connected : Consumer
    , 'TB4S001400CS' #TVS Now Gross Adds
    ]

ga_monthly_df = chk_src_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
ga_monthly_df = ga_monthly_df.loc[ga_monthly_df['METRIC_CD'].isin(v_metric_list)]
ga_monthly_df = ga_monthly_df.loc[ga_monthly_df['TM_KEY_MTH']>=v_tm_key_mth]

ga_monthly_df['PRE_GA'] = np.where(ga_monthly_df['METRIC_CD']=='B1S000101CS', ga_monthly_df['P'], 0)
ga_monthly_df['POST_GA_B2C'] = np.where(ga_monthly_df['METRIC_CD']=='B2S010100CS', ga_monthly_df['P'], 0)
ga_monthly_df['TOL_GA_CONN_CON'] = np.where(ga_monthly_df['METRIC_CD']=='TB3S000102CS', ga_monthly_df['P'], 0)
ga_monthly_df['TVS_NOW_GA'] = np.where(ga_monthly_df['METRIC_CD']=='TB4S001400CS', ga_monthly_df['P'], 0)

ga_monthly_df = ga_monthly_df.groupby('TM_KEY_MTH').agg({'PPN_TM':'max', 'PRE_GA':'sum', 'POST_GA_B2C':'sum', 'TOL_GA_CONN_CON':'sum', 'TVS_NOW_GA':'sum'})
ga_monthly_df = ga_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
ga_monthly_df = ga_monthly_df[['TM_KEY_MTH', 'PPN_TM', 'PRE_GA', 'POST_GA_B2C', 'TOL_GA_CONN_CON', 'TVS_NOW_GA']]

mod_col_list = ga_monthly_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    ga_monthly_df[col] = ga_monthly_df[col].apply(lambda x: format(x, ',.0f'))
ga_monthly_df

,TM_KEY_MTH,PPN_TM,PRE_GA,POST_GA_B2C,TOL_GA_CONN_CON,TVS_NOW_GA
0,202506,2025-06-20 00:48:43,"978,914","106,466","31,455","1,112"


### Weekly

In [19]:
''' Gross Adds Weekly '''

v_metric_list = [
    'B1S000101CS' #Prepaid Gross Adds
    , 'B2S010100CS' #Postpaid Gross Adds B2C
    , 'TB3S000102CS' #TOL Gross Adds Connected : Consumer
    , 'TB4S001400CS' #TVS Now Gross Adds
    ]

ga_weekly_df = chk_src_df[['TM_KEY_WK', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
ga_weekly_df = ga_weekly_df.loc[ga_weekly_df['METRIC_CD'].isin(v_metric_list)]
# ga_weekly_df = ga_weekly_df.loc[ga_weekly_df['TM_KEY_WK']>=v_tm_key_wk]

ga_weekly_df['PRE_GA'] = np.where(ga_weekly_df['METRIC_CD']=='B1S000101CS', ga_weekly_df['P'], 0)
ga_weekly_df['POST_GA_B2C'] = np.where(ga_weekly_df['METRIC_CD']=='B2S010100CS', ga_weekly_df['P'], 0)
ga_weekly_df['TOL_GA_CONN_CON'] = np.where(ga_weekly_df['METRIC_CD']=='TB3S000102CS', ga_weekly_df['P'], 0)
ga_weekly_df['TVS_NOW_GA'] = np.where(ga_weekly_df['METRIC_CD']=='TB4S001400CS', ga_weekly_df['P'], 0)

ga_weekly_df = ga_weekly_df.groupby('TM_KEY_WK').agg({'TM_KEY_DAY':'max', 'PPN_TM':'max', 'PRE_GA':'sum', 'POST_GA_B2C':'sum', 'TOL_GA_CONN_CON':'sum', 'TVS_NOW_GA':'sum'})
ga_weekly_df = ga_weekly_df.fillna(0).sort_values(by=['TM_KEY_WK']).reset_index()
ga_weekly_df = ga_weekly_df[['TM_KEY_WK', 'TM_KEY_DAY', 'PPN_TM', 'PRE_GA', 'POST_GA_B2C', 'TOL_GA_CONN_CON', 'TVS_NOW_GA']]
ga_weekly_df.rename(columns={'TM_KEY_DAY':'AS_DAY'}, inplace=True)

mod_col_list = ga_weekly_df.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    ga_weekly_df[col] = ga_weekly_df[col].apply(lambda x: format(x, ',.0f'))
ga_weekly_df

,TM_KEY_WK,AS_DAY,PPN_TM,PRE_GA,POST_GA_B2C,TOL_GA_CONN_CON,TVS_NOW_GA
0,2025022,20250601,2025-06-20 00:48:43,"64,629","5,571","1,670",81
1,2025023,20250608,2025-06-20 00:48:43,"421,599","42,351","12,379",550
2,2025024,20250615,2025-06-20 00:48:43,"387,119","40,539","11,599",375
3,2025025,20250619,2025-06-20 00:48:43,"105,567","18,005","5,807",106


### Daily

In [20]:
''' Gross Adds Daily '''

v_metric_list = [
    'B1S000101CS' #Prepaid Gross Adds
    , 'B2S010100CS' #Postpaid Gross Adds B2C
    , 'TB3S000102CS' #TOL Gross Adds Connected : Consumer
    , 'TB4S001400CS' #TVS Now Gross Adds
    ]

ga_daily_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
ga_daily_df = ga_daily_df.loc[ga_daily_df['METRIC_CD'].isin(v_metric_list)]
ga_daily_df = ga_daily_df.loc[ga_daily_df['TM_KEY_MTH']>=v_tm_key_mth]

ga_daily_df['PRE_GA'] = np.where(ga_daily_df['METRIC_CD']=='B1S000101CS', ga_daily_df['P'], 0)
ga_daily_df['POST_GA_B2C'] = np.where(ga_daily_df['METRIC_CD']=='B2S010100CS', ga_daily_df['P'], 0)
ga_daily_df['TOL_GA_CONN_CON'] = np.where(ga_daily_df['METRIC_CD']=='TB3S000102CS', ga_daily_df['P'], 0)
ga_daily_df['TVS_NOW_GA'] = np.where(ga_daily_df['METRIC_CD']=='TB4S001400CS', ga_daily_df['P'], 0)

ga_daily_df = ga_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max', 'PRE_GA':'sum', 'POST_GA_B2C':'sum', 'TOL_GA_CONN_CON':'sum', 'TVS_NOW_GA':'sum'})
ga_daily_df = ga_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY'], ascending=False).reset_index()
ga_daily_df = ga_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'PRE_GA', 'POST_GA_B2C', 'TOL_GA_CONN_CON', 'TVS_NOW_GA']]

mod_col_list = ga_daily_df.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    ga_daily_df[col] = ga_daily_df[col].apply(lambda x: format(x, ',.0f'))
ga_daily_df

,TM_KEY_MTH,TM_KEY_DAY,PPN_TM,PRE_GA,POST_GA_B2C,TOL_GA_CONN_CON,TVS_NOW_GA
0,202506,20250619,2025-06-20 00:48:43,0,0,0,0
1,202506,20250618,2025-06-20 00:48:43,0,"5,394","1,867",0
2,202506,20250617,2025-06-20 00:48:43,"51,524","6,994","1,916",57
3,202506,20250616,2025-06-20 00:48:43,"54,043","5,617","2,024",49
4,202506,20250615,2025-06-20 00:48:43,"51,015","5,003","1,369",44
5,202506,20250614,2025-06-20 00:48:43,"53,060","5,674","1,831",44
6,202506,20250613,2025-06-20 00:48:43,"55,207","5,571","1,538",46
7,202506,20250612,2025-06-20 00:48:43,"56,890","5,598","1,647",54
8,202506,20250611,2025-06-20 00:48:43,"55,054","5,894","1,641",39
9,202506,20250610,2025-06-20 00:48:43,"60,156","6,857","1,785",71


## Churn Subs by Period

### Yearly

In [21]:
''' Churn Subs Yearly '''

v_metric_list = [
    'B1S000200' #Prepaid Churn Subs
    , 'B2S010200' #Postpaid Churn Subs B2C
    # , 'B2S010201' #Postpaid Churn Subs Voluntary B2C
    # , 'B2S010202' #Postpaid Churn Subs Involuntary B2C
    , 'TSER13100' #FTTx Churn Subs
    , 'TSER13102' #FTTx Churn Subs Voluntary
    , 'TSER13103' #FTTx Churn Subs Involuntary
    , 'TSER13104' #FTTx Churn Subs Involuntary (Collection Forecast)
    , 'TSER14100' #TVS Churn Subs
    # , 'TSER14102' #TVS Churn Subs Voluntary
    # , 'TSER14103' #TVS Churn Subs Involuntary
    ]

churn_yearly_df = chk_src_df[['TM_KEY_YR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
churn_yearly_df = churn_yearly_df.loc[churn_yearly_df['METRIC_CD'].isin(v_metric_list)]

churn_yearly_df['PRE_CHURN'] = np.where(churn_yearly_df['METRIC_CD']=='B1S000200', churn_yearly_df['P'], 0)
churn_yearly_df['POST_CHURN_B2C'] = np.where(churn_yearly_df['METRIC_CD']=='B2S010200', churn_yearly_df['P'], 0)
churn_yearly_df['FTTX_CHURN'] = np.where(churn_yearly_df['METRIC_CD']=='TSER13100', churn_yearly_df['P'], 0)
churn_yearly_df['FTTX_VOLUN'] = np.where(churn_yearly_df['METRIC_CD']=='TSER13102', churn_yearly_df['P'], 0)
churn_yearly_df['FTTX_INVOLUN'] = np.where(churn_yearly_df['METRIC_CD']=='TSER13103', churn_yearly_df['P'], 0)
churn_yearly_df['FTTX_INVOLUN(CF)'] = np.where(churn_yearly_df['METRIC_CD']=='TSER13104', churn_yearly_df['P'], 0)
churn_yearly_df['TVS_CHURN'] = np.where(churn_yearly_df['METRIC_CD']=='TSER14100', churn_yearly_df['P'], 0)

churn_yearly_df = churn_yearly_df.groupby('TM_KEY_YR').agg({'PPN_TM':'max', 'PRE_CHURN':'sum', 'POST_CHURN_B2C':'sum', 'FTTX_CHURN':'sum', 'FTTX_VOLUN':'sum', 'FTTX_INVOLUN':'sum', 'FTTX_INVOLUN(CF)':'sum', 'TVS_CHURN':'sum'})
churn_yearly_df = churn_yearly_df.fillna(0).sort_values(by=['TM_KEY_YR']).reset_index()
churn_yearly_df = churn_yearly_df[['TM_KEY_YR', 'PPN_TM', 'PRE_CHURN', 'POST_CHURN_B2C', 'FTTX_CHURN', 'FTTX_VOLUN', 'FTTX_INVOLUN', 'FTTX_INVOLUN(CF)', 'TVS_CHURN']]

mod_col_list = churn_yearly_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    churn_yearly_df[col] = churn_yearly_df[col].apply(lambda x: format(x, ',.0f'))
churn_yearly_df

,TM_KEY_YR,PPN_TM,PRE_CHURN,POST_CHURN_B2C,FTTX_CHURN,FTTX_VOLUN,FTTX_INVOLUN,FTTX_INVOLUN(CF),TVS_CHURN
0,2024,2025-06-20 00:48:43,"34,112,179","2,857,216","456,668","170,154","281,222","282,188","194,250"
1,2025,2025-06-20 00:48:43,"29,971,763","1,053,745","193,981","73,892","125,678","174,030","102,645"


### Quarterly

In [22]:
''' Churn Subs Quarterly '''

v_metric_list = [
    'B1S000200' #Prepaid Churn Subs
    , 'B2S010200' #Postpaid Churn Subs B2C
    # , 'B2S010201' #Postpaid Churn Subs Voluntary B2C
    # , 'B2S010202' #Postpaid Churn Subs Involuntary B2C
    , 'TSER13100' #FTTx Churn Subs
    , 'TSER13102' #FTTx Churn Subs Voluntary
    , 'TSER13103' #FTTx Churn Subs Involuntary
    , 'TSER13104' #FTTx Churn Subs Involuntary (Collection Forecast)
    , 'TSER14100' #TVS Churn Subs
    # , 'TSER14102' #TVS Churn Subs Voluntary
    # , 'TSER14103' #TVS Churn Subs Involuntary
    ]

churn_quarterly_df = chk_src_df[['TM_KEY_QTR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
churn_quarterly_df = churn_quarterly_df.loc[churn_quarterly_df['METRIC_CD'].isin(v_metric_list)]
churn_quarterly_df = churn_quarterly_df.loc[churn_quarterly_df['TM_KEY_QTR']>=v_tm_key_qtr]

churn_quarterly_df['PRE_CHURN'] = np.where(churn_quarterly_df['METRIC_CD']=='B1S000200', churn_quarterly_df['P'], 0)
churn_quarterly_df['POST_CHURN_B2C'] = np.where(churn_quarterly_df['METRIC_CD']=='B2S010200', churn_quarterly_df['P'], 0)
churn_quarterly_df['FTTX_CHURN'] = np.where(churn_quarterly_df['METRIC_CD']=='TSER13100', churn_quarterly_df['P'], 0)
churn_quarterly_df['FTTX_VOLUN'] = np.where(churn_quarterly_df['METRIC_CD']=='TSER13102', churn_quarterly_df['P'], 0)
churn_quarterly_df['FTTX_INVOLUN'] = np.where(churn_quarterly_df['METRIC_CD']=='TSER13103', churn_quarterly_df['P'], 0)
churn_quarterly_df['FTTX_INVOLUN(CF)'] = np.where(churn_quarterly_df['METRIC_CD']=='TSER13104', churn_quarterly_df['P'], 0)
churn_quarterly_df['TVS_CHURN'] = np.where(churn_quarterly_df['METRIC_CD']=='TSER14100', churn_quarterly_df['P'], 0)

churn_quarterly_df = churn_quarterly_df.groupby('TM_KEY_QTR').agg({'PPN_TM':'max', 'PRE_CHURN':'sum', 'POST_CHURN_B2C':'sum', 'FTTX_CHURN':'sum', 'FTTX_VOLUN':'sum', 'FTTX_INVOLUN':'sum', 'FTTX_INVOLUN(CF)':'sum', 'TVS_CHURN':'sum'})
churn_quarterly_df = churn_quarterly_df.fillna(0).sort_values(by=['TM_KEY_QTR']).reset_index()
churn_quarterly_df = churn_quarterly_df[['TM_KEY_QTR', 'PPN_TM', 'PRE_CHURN', 'POST_CHURN_B2C', 'FTTX_CHURN', 'FTTX_VOLUN', 'FTTX_INVOLUN', 'FTTX_INVOLUN(CF)', 'TVS_CHURN']]

mod_col_list = churn_quarterly_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    churn_quarterly_df[col] = churn_quarterly_df[col].apply(lambda x: format(x, ',.0f'))
churn_quarterly_df

,TM_KEY_QTR,PPN_TM,PRE_CHURN,POST_CHURN_B2C,FTTX_CHURN,FTTX_VOLUN,FTTX_INVOLUN,FTTX_INVOLUN(CF),TVS_CHURN
0,20241,2025-06-20 00:48:43,"9,491,554","871,395","118,742","43,223","72,945","73,317","48,913"
1,20242,2025-06-20 00:48:43,"8,344,376","718,051","114,261","43,677","70,807","70,972","40,970"
2,20243,2025-06-20 00:48:43,"8,899,074","664,232","118,872","44,269","67,909","68,985","48,795"
3,20244,2025-06-20 00:48:43,"7,377,175","603,538","104,793","38,985","69,561","68,914","55,572"
4,20251,2025-06-20 00:48:43,"7,728,433","553,494","104,815","38,083","68,600","68,746","47,799"
5,20252,2025-06-20 00:48:43,"22,243,330","500,251","89,166","35,809","57,078","105,284","54,846"


### Monthly

In [23]:
''' Churn Subs Monthly '''

v_metric_list = [
    'B1S000200' #Prepaid Churn Subs
    , 'B2S010200' #Postpaid Churn Subs B2C
    # , 'B2S010201' #Postpaid Churn Subs Voluntary B2C
    # , 'B2S010202' #Postpaid Churn Subs Involuntary B2C
    , 'TSER13100' #FTTx Churn Subs
    , 'TSER13102' #FTTx Churn Subs Voluntary
    , 'TSER13103' #FTTx Churn Subs Involuntary
    , 'TSER13104' #FTTx Churn Subs Involuntary (Collection Forecast)
    , 'TSER14100' #TVS Churn Subs
    # , 'TSER14102' #TVS Churn Subs Voluntary
    # , 'TSER14103' #TVS Churn Subs Involuntary
    ]

churn_monthly_df = chk_src_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
churn_monthly_df = churn_monthly_df.loc[churn_monthly_df['METRIC_CD'].isin(v_metric_list)]
churn_monthly_df = churn_monthly_df.loc[churn_monthly_df['TM_KEY_MTH']>=v_tm_key_mth]

churn_monthly_df['PRE_CHURN'] = np.where(churn_monthly_df['METRIC_CD']=='B1S000200', churn_monthly_df['P'], 0)
churn_monthly_df['POST_CHURN_B2C'] = np.where(churn_monthly_df['METRIC_CD']=='B2S010200', churn_monthly_df['P'], 0)
churn_monthly_df['FTTX_CHURN'] = np.where(churn_monthly_df['METRIC_CD']=='TSER13100', churn_monthly_df['P'], 0)
churn_monthly_df['FTTX_VOLUN'] = np.where(churn_monthly_df['METRIC_CD']=='TSER13102', churn_monthly_df['P'], 0)
churn_monthly_df['FTTX_INVOLUN'] = np.where(churn_monthly_df['METRIC_CD']=='TSER13103', churn_monthly_df['P'], 0)
churn_monthly_df['FTTX_INVOLUN(CF)'] = np.where(churn_monthly_df['METRIC_CD']=='TSER13104', churn_monthly_df['P'], 0)
churn_monthly_df['TVS_CHURN'] = np.where(churn_monthly_df['METRIC_CD']=='TSER14100', churn_monthly_df['P'], 0)

churn_monthly_df = churn_monthly_df.groupby('TM_KEY_MTH').agg({'PPN_TM':'max', 'PRE_CHURN':'sum', 'POST_CHURN_B2C':'sum', 'FTTX_CHURN':'sum', 'FTTX_VOLUN':'sum', 'FTTX_INVOLUN':'sum', 'FTTX_INVOLUN(CF)':'sum', 'TVS_CHURN':'sum'})
churn_monthly_df = churn_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
churn_monthly_df = churn_monthly_df[['TM_KEY_MTH', 'PPN_TM', 'PRE_CHURN', 'POST_CHURN_B2C', 'FTTX_CHURN', 'FTTX_VOLUN', 'FTTX_INVOLUN', 'FTTX_INVOLUN(CF)', 'TVS_CHURN']]

mod_col_list = churn_monthly_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    churn_monthly_df[col] = churn_monthly_df[col].apply(lambda x: format(x, ',.0f'))
churn_monthly_df

,TM_KEY_MTH,PPN_TM,PRE_CHURN,POST_CHURN_B2C,FTTX_CHURN,FTTX_VOLUN,FTTX_INVOLUN,FTTX_INVOLUN(CF),TVS_CHURN
0,202506,2025-06-20 00:48:43,"-2,502,714","114,370","26,388","8,122","10,395","24,435","15,190"


### Weekly

In [24]:
''' Churn Subs Weekly '''

v_metric_list = [
    'B1S000200' #Prepaid Churn Subs
    , 'B2S010200' #Postpaid Churn Subs B2C
    # , 'B2S010201' #Postpaid Churn Subs Voluntary B2C
    # , 'B2S010202' #Postpaid Churn Subs Involuntary B2C
    , 'TSER13100' #FTTx Churn Subs
    , 'TSER13102' #FTTx Churn Subs Voluntary
    , 'TSER13103' #FTTx Churn Subs Involuntary
    , 'TSER13104' #FTTx Churn Subs Involuntary (Collection Forecast)
    , 'TSER14100' #TVS Churn Subs
    # , 'TSER14102' #TVS Churn Subs Voluntary
    # , 'TSER14103' #TVS Churn Subs Involuntary
    ]

churn_weekly_df = chk_src_df[['TM_KEY_WK', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
churn_weekly_df = churn_weekly_df.loc[churn_weekly_df['METRIC_CD'].isin(v_metric_list)]
# churn_weekly_df = churn_weekly_df.loc[churn_weekly_df['TM_KEY_WK']>=v_tm_key_wk]

churn_weekly_df['PRE_CHURN'] = np.where(churn_weekly_df['METRIC_CD']=='B1S000200', churn_weekly_df['P'], 0)
churn_weekly_df['POST_CHURN_B2C'] = np.where(churn_weekly_df['METRIC_CD']=='B2S010200', churn_weekly_df['P'], 0)
churn_weekly_df['FTTX_CHURN'] = np.where(churn_weekly_df['METRIC_CD']=='TSER13100', churn_weekly_df['P'], 0)
churn_weekly_df['FTTX_VOLUN'] = np.where(churn_weekly_df['METRIC_CD']=='TSER13102', churn_weekly_df['P'], 0)
churn_weekly_df['FTTX_INVOLUN'] = np.where(churn_weekly_df['METRIC_CD']=='TSER13103', churn_weekly_df['P'], 0)
churn_weekly_df['FTTX_INVOLUN(CF)'] = np.where(churn_weekly_df['METRIC_CD']=='TSER13104', churn_weekly_df['P'], 0)
churn_weekly_df['TVS_CHURN'] = np.where(churn_weekly_df['METRIC_CD']=='TSER14100', churn_weekly_df['P'], 0)

churn_weekly_df = churn_weekly_df.groupby('TM_KEY_WK').agg({'TM_KEY_DAY':'max', 'PPN_TM':'max', 'PRE_CHURN':'sum', 'POST_CHURN_B2C':'sum', 'FTTX_CHURN':'sum', 'FTTX_VOLUN':'sum', 'FTTX_INVOLUN':'sum', 'FTTX_INVOLUN(CF)':'sum', 'TVS_CHURN':'sum'})
churn_weekly_df = churn_weekly_df.fillna(0).sort_values(by=['TM_KEY_WK']).reset_index()
churn_weekly_df = churn_weekly_df[['TM_KEY_WK', 'TM_KEY_DAY', 'PPN_TM', 'PRE_CHURN', 'POST_CHURN_B2C', 'FTTX_CHURN', 'FTTX_VOLUN', 'FTTX_INVOLUN', 'FTTX_INVOLUN(CF)', 'TVS_CHURN']]
churn_weekly_df.rename(columns={'TM_KEY_DAY':'AS_DAY'}, inplace=True)

mod_col_list = churn_weekly_df.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    churn_weekly_df[col] = churn_weekly_df[col].apply(lambda x: format(x, ',.0f'))
churn_weekly_df

,TM_KEY_WK,AS_DAY,PPN_TM,PRE_CHURN,POST_CHURN_B2C,FTTX_CHURN,FTTX_VOLUN,FTTX_INVOLUN,FTTX_INVOLUN(CF),TVS_CHURN
0,2025022,20250601,2025-06-20 00:48:43,"67,669","4,299",-814,641,0,"24,435","1,142"
1,2025023,20250608,2025-06-20 00:48:43,"4,683,931","49,654","14,695","3,956","1,905",0,"6,653"
2,2025024,20250615,2025-06-20 00:48:43,"-7,465,257","48,846","12,507","3,525","8,490",0,"5,858"
3,2025025,20250619,2025-06-20 00:48:43,"210,943","11,571",0,0,0,0,"1,537"


### Daily

In [25]:
''' Churn Subs Daily '''

v_metric_list = [
    'B1S000200' #Prepaid Churn Subs
    , 'B2S010200' #Postpaid Churn Subs B2C
    # , 'B2S010201' #Postpaid Churn Subs Voluntary B2C
    # , 'B2S010202' #Postpaid Churn Subs Involuntary B2C
    , 'TSER13100' #FTTx Churn Subs
    , 'TSER13102' #FTTx Churn Subs Voluntary
    , 'TSER13103' #FTTx Churn Subs Involuntary
    , 'TSER13104' #FTTx Churn Subs Involuntary (Collection Forecast)
    , 'TSER14100' #TVS Churn Subs
    # , 'TSER14102' #TVS Churn Subs Voluntary
    # , 'TSER14103' #TVS Churn Subs Involuntary
    ]

churn_daily_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
churn_daily_df = churn_daily_df.loc[churn_daily_df['METRIC_CD'].isin(v_metric_list)]
churn_daily_df = churn_daily_df.loc[churn_daily_df['TM_KEY_MTH']>=v_tm_key_mth]

churn_daily_df['PRE_CHURN'] = np.where(churn_daily_df['METRIC_CD']=='B1S000200', churn_daily_df['P'], 0)
churn_daily_df['POST_CHURN_B2C'] = np.where(churn_daily_df['METRIC_CD']=='B2S010200', churn_daily_df['P'], 0)
churn_daily_df['FTTX_CHURN'] = np.where(churn_daily_df['METRIC_CD']=='TSER13100', churn_daily_df['P'], 0)
churn_daily_df['FTTX_VOLUN'] = np.where(churn_daily_df['METRIC_CD']=='TSER13102', churn_daily_df['P'], 0)
churn_daily_df['FTTX_INVOLUN'] = np.where(churn_daily_df['METRIC_CD']=='TSER13103', churn_daily_df['P'], 0)
churn_daily_df['FTTX_INVOLUN(CF)'] = np.where(churn_daily_df['METRIC_CD']=='TSER13104', churn_daily_df['P'], 0)
churn_daily_df['TVS_CHURN'] = np.where(churn_daily_df['METRIC_CD']=='TSER14100', churn_daily_df['P'], 0)

churn_daily_df = churn_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max', 'PRE_CHURN':'sum', 'POST_CHURN_B2C':'sum', 'FTTX_CHURN':'sum', 'FTTX_VOLUN':'sum', 'FTTX_INVOLUN':'sum', 'FTTX_INVOLUN(CF)':'sum', 'TVS_CHURN':'sum'})
churn_daily_df = churn_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY'], ascending=False).reset_index()
churn_daily_df = churn_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'PRE_CHURN', 'POST_CHURN_B2C', 'FTTX_CHURN', 'FTTX_VOLUN', 'FTTX_INVOLUN', 'FTTX_INVOLUN(CF)', 'TVS_CHURN']]

mod_col_list = churn_daily_df.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    churn_daily_df[col] = churn_daily_df[col].apply(lambda x: format(x, ',.0f'))
churn_daily_df

,TM_KEY_MTH,TM_KEY_DAY,PPN_TM,PRE_CHURN,POST_CHURN_B2C,FTTX_CHURN,FTTX_VOLUN,FTTX_INVOLUN,FTTX_INVOLUN(CF),TVS_CHURN
0,202506,20250619,2025-06-20 00:48:43,0,0,0,0,0,0,0
1,202506,20250618,2025-06-20 00:48:43,"62,715",0,0,0,0,0,0
2,202506,20250617,2025-06-20 00:48:43,"74,854","7,804",0,0,0,0,728
3,202506,20250616,2025-06-20 00:48:43,"73,374","3,767",0,0,0,0,809
4,202506,20250615,2025-06-20 00:48:43,"69,818","7,935","4,887",425,0,0,967
5,202506,20250614,2025-06-20 00:48:43,"66,912","7,048",-513,515,0,0,751
6,202506,20250613,2025-06-20 00:48:43,"62,866","4,491",-807,467,"4,210",0,806
7,202506,20250612,2025-06-20 00:48:43,"63,109","3,050","4,661",503,0,0,808
8,202506,20250611,2025-06-20 00:48:43,"-3,899,168","4,385",-575,544,0,0,814
9,202506,20250610,2025-06-20 00:48:43,"-3,908,637","16,100",-468,485,0,0,816
